In [ ]:
import kfp
import kfp.dsl as dsl

@dsl.pipeline(
    name='MnistPipeline',
    description='mnist '
)
def mnist_katib_pipeline(user_namespace, max_failed_trial_count, parallel_trial_count, max_trial_count, train_image_name):
        
    mnist_katib_job = dsl.ContainerOp(
        name='mnist_katib_job',
        image='kubeflow-registry.default.svc.cluster.local:30000/mnist-katib-job:91DE0588',
        command=['python', '/app/mnist_for_katib.py'],
        arguments=[
            "--user_namespace", user_namespace,
            "--max_failed_trial_count", max_failed_trial_count,
            "--parallel_trial_count", parallel_trial_count,
            "--max_trial_count", max_trial_count,
            "--train_image_name", train_image_name
        ],
        file_outputs={
            'output': '/output.txt',
        },            
        output_artifact_paths={'mlpipeline-metrics': '/mlpipeline-metrics.json'},
    ).output
    
    
    monitor_for_katib = dsl.ContainerOp(
        name='monitor_for_katib_job',
        image='kubeflow-registry.default.svc.cluster.local:30000/monitor-katib-job:D06ACD15',
        command=['python', "-u", '/app/monitor_for_katib.py'],
        arguments=[
            "--experiment_name", mnist_katib_job,
            "--namespace", user_namespace
        ]
    )
    

        
arguments = {'user_namespace': 'amaramusic',
             'max_failed_trial_count': '3',
             'parallel_trial_count': '1',
             'max_trial_count': '12',
             'train_image_name' : 'kubeflow-registry.default.svc.cluster.local:30000/mnist-train:C3081A64'
            }

if __name__ == '__main__':
    kfp.Client().create_run_from_pipeline_func(pipeline_func=mnist_katib_pipeline, 
                                               arguments=arguments)

In [ ]:
!pip install kubeflow-katib==0.0.2